In [110]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(url, names=names)
transform = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
dataset["class"] = dataset["class"].apply(lambda x: transform[x])

# Загружаем данные

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
X_train = pd.read_csv("./UCI HAR Dataset/train/X_train.txt", sep=",", names=[i for i in range(0,561)]).to_numpy()
X_test = pd.read_csv("./UCI HAR Dataset/test/X_test.txt", sep=",", names=[i for i in range(0,561)]).to_numpy()
y_train = pd.read_csv("./UCI HAR Dataset/train/y_train.txt", names=["Y"]).to_numpy()
y_test = pd.read_csv("./UCI HAR Dataset/test/y_test.txt", names=["Y"]).to_numpy()

In [190]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Класс, который в последствии добавляется в словарь для удобного выбора
class Node:
    def __init__(
        self,
        feature=None,
        threshold=None,
        # childs=None,
        left = None,
        right = None,
        value=None
    ):
        self.feature = feature
        self.threshold = threshold
        # self.childs = childs
        self.value = value
        self.right = right
        self.left = left

    def is_leaf_node(self):
        return self.value is not None
    


class DecisionTree:
    def __init__(self, max_depth=10, min_samples=10):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None


    def fit(self, X, y):
        self.tree = self.grow_tree(X, y)

    def calck_unic(self, a: list):
        keys = np.unique(a)
        # return {key: a[a == key].shape[0] for key in keys}
        return (a[a == key].shape[0] for key in keys)

    def predict(self, X):
        return np.array([self.travers_tree(x, self.tree) for x in X])

    def entropy(self, y: np.ndarray):
        hist = self.calck_unic(y)
        n = y.shape[0]
        # hist = {key: val / len(y) for key, val in hist.items()}
        info = -np.sum(np.fromiter((val/n * np.log2(val/n) for val in hist), dtype=np.float32))

        return info
    
    #======================================
    def gini(self, y:np.ndarray):
        uitems = self.calck_unic(y)
        n = y.shape[0]
        return 1 - np.sum(np.fromiter(((val/n)**2 for val in uitems), dtype=np.float32))
    #======================================

    def information_gain(self, X_column: list, y: list, threshold:float):
        if np.unique(y).shape[0] == 1:
            return 0

        n = y.shape[0]
        # info(T)
        # parent = self.entropy(y)
        # uitems = self.calck_unic(X_column)
        #======================================
        left_inds = np.argwhere(X_column <= threshold).flatten()
        right_inds = np.argwhere(X_column > threshold).flatten()
        # print(left_inds)
        # print(X_column.shape)
        # print(y.shape)
        gini_left = self.gini(y[left_inds])
        gini_right = self.gini(y[right_inds])
        gini_split = (len(left_inds) / n) * gini_left +  (len(right_inds) / n) * gini_right
        return gini_split, threshold
        #====================================== 
        # info_x = np.sum(
        #     [val / n * self.entropy(y[X_column == key]) for key, val in uitems.items()]
        # )
        # split_info = -np.sum(
        #     val / n * np.log2(val / n) for val in uitems.values() if val > 0
        # )
        # if split_info != 0:
        #     return (parent - info_x) / split_info, list(uitems.keys())
        # else:
        #     return 0, list(uitems.keys())

    def most_common(self, y):
        labels = np.unique(y)
        vals = self.calck_unic(y)
        return labels[np.argmax(vals)]

    def best_split(self, X, y):
        best_feature, best_threshold = None, None
        # best_gain = -1
        best_gini = 1000000
        # uitems = []
        for i in range(X.shape[1]):
            #==============================================
            trasholds = np.random.choice(np.unique(X[:, i]), 10)
            # print("new_i")
            for trashold in trasholds:
            #==============================================
                gini, threshold = self.information_gain(X[:, i], y, trashold)
                if gini < best_gini:
                    best_gini = gini
                    best_feature = i
                    best_threshold = threshold

        return best_feature, best_threshold

    def grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = np.unique(y).shape[0]
        # print("depth1:", depth)
        if n_samples <= self.min_samples or depth >= self.max_depth or n_labels <= 1:
            return Node(value=self.most_common(y))

        #==============================================
        best_feature, best_threshold = self.best_split(X, y)
        l_inds = np.argwhere(X[:, best_feature] <= best_threshold).flatten()
        r_inds = np.argwhere(X[:, best_feature] > best_threshold).flatten()
        # print("depth2:", depth)
        if len(l_inds) == 0 or len(r_inds) == 0:
            return Node(value=self.most_common(y))
        
        # print("depth3:", depth)
        left = self.grow_tree(X = X[l_inds], y = y[l_inds], depth = depth+1)
        right = self.grow_tree(X = X[r_inds], y = y[r_inds], depth = depth+1)

        return Node(best_feature, best_threshold, left, right)
        #==============================================

        # В словаре содержатся не словари, а Node По сути, словарь содержит ссылки на объекты, а нужен он для более удобной навигации.
        # childs = {
        #     key: self.grow_tree(
        #         X[X[:, best_feature] == key],
        #         y[X[:, best_feature] == key],
        #         depth=depth + 1,
        #     )
        #     for key in ukeys
        # }

        # return Node(best_feature, childs=childs)

    #==============================================
    def travers_tree(self, x, tree):
        if tree.is_leaf_node():
            return tree.value
        
        return self.travers_tree(x, tree.left) if tree.threshold >= x[tree.feature] else self.travers_tree(x, tree.right)
    #==============================================

    # def travers_tree(self, x, tree):
    #     if tree.is_leaf_node():
    #         return tree.value

    #     return self.travers_tree(
    #         x,
    #         tree.childs.get(
    #             x[tree.feature], tree.childs.get(list(tree.childs.keys())[0])
    #         ),
    #     )


def preload():
    df = pd.read_csv("agaricus-lepiota.csv")
    y = np.array(df.pop("poison"))

    n = df.shape[1]
    to_del = int(n ** 0.5)
    import random

    for i in range(n - to_del - 1):
        df.pop(random.choice(df.columns))
    x = np.array(df)
    return x, y


def acc_pre_rec(otv, y):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for i in range(len(otv)):
        if otv[i] == "p" and y[i] == "p":
            tp += 1
        elif otv[i] == "p" and y[i] == "e":
            fp += 1
        elif otv[i] == "e" and y[i] == "p":
            fn += 1
        elif otv[i] == "e" and y[i] == "e":
            tn += 1

    acc = (tp + tn) / (tp + tn + fp + fn)
    pre = tp / max((tp + fp), 1)
    rec = tp / max(tp + fn, 1)
    fpr = fp / max(tn + fp, 1)

    return acc, pre, rec, fpr

# x, y = preload()
# print(len(y))


# x = dataset[dataset.columns[:-1]].to_numpy()[:, :2]
# y = dataset["class"].to_numpy()


# clf = DecisionTree(max_depth=2)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
# clf.fit(x_train, y_train)

# otv = clf.predict(x_test)

# print(f1_score(y_test, otv, average="micro"))

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
# clf.fit(x_train, y_train)

# otv = clf.predict(x_test)
# acc, pre, rec, fpr = acc_pre_rec(y_test, otv)
# print(f"accuracy: {acc}")
# print(f"precision: {pre}")
# print(f"recall: {rec} ")

# points = np.array([[0, 0], [fpr, rec], [1, 1]])
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 7))

# plt.plot(
#     points[:, 0],
#     points[:, 1],
#     "o-r",
#     alpha=0.7,
#     label="first",
#     lw=5,
#     mec="b",
#     mew=2,
#     ms=10,
# )
# plt.fill_between(points[:, 0],
#     points[:, 1], where=None, interpolate=False, step=None)

In [192]:
class GradientBoosting:
    def __init__(self, learning_rate=0.2, n_estimators=100, max_depth=3):
        """
        :param learning_rate: скорость обучения
        :param n_estimators: количество деревьев
        :param max_depth: максимальная глубина деревьев
        """
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.num_classes = None
        self.base_estimators = []
        
    def sigmoid(self, y_pred):
        """
        :param y_pred: прогноз модели
        :return: значение функции сигмоиды для прогноза
        """
        return 1.0 / (1.0 + np.exp(-y_pred))
    
    def softmax(self, y_pred):
        """
        :param y_pred: прогноз модели
        :return: значение функции softmax для прогноза
        """
        exp_pred = np.exp(y_pred)
        return exp_pred / np.sum(exp_pred, axis=1)[:, np.newaxis]
    
    def logloss_gradient(self, y_true, y_pred):
        """
        :param y_true: истинный ответ
        :param y_pred: прогноз модели
        :return: значение градиента функции потерь logLoss
        """
        # p = self.softmax(y_pred)
        return (y_pred - y_true) / y_true.shape[0]
    
    def fit(self, X, y):
        """
        Обучение градиентного бустинга с OneVsAll методом.
        :param X: матрица объектов (numpy array)
        :param y: вектор истинных ответов (numpy array)
        """
        self.num_classes = np.unique(y).shape[0]

        for k in range(self.num_classes):
            y_k = np.where(y == k, 1, 0)
            # print(y_k)
            # print(y_k)
            # print(y)
            y_pred = np.zeros_like(y_k, dtype=np.float64)
            estimator = DecisionTree(max_depth=self.max_depth)
            estimator.fit(X, y_k)
            y_pred += estimator.predict(X)
            self.base_estimators.append((estimator, k))
            # print(y_pred)
            for m in range(self.n_estimators - 1):
                grad = self.logloss_gradient(y_k, y_pred)
                # print(grad)
                estimator = DecisionTree(max_depth=self.max_depth)
                # Идём в сторону АНТИГРАДИЕНТА БЛИН, АН-ТИ ГРАДИЕНТА (-2 часа в попытках найти баг в дереве)
                estimator.fit(X, -grad)
                self.base_estimators.append((estimator, k))
                otv = estimator.predict(X)
                y_pred += self.learning_rate * otv
                
                
    def predict_proba(self, X, class_check:int = None):
        """
        Предсказание вероятности класса.
        """
        if not class_check:
            y_pred_proba = np.zeros((X.shape[0], self.num_classes))
        else:
            y_pred_proba = np.zeros((X.shape[0], 2))

        # print(y_pred_proba)
        for m in range(self.n_estimators):
            flag = 1

            for estimator, k in self.base_estimators:
                if not class_check:
                    y_pred_proba[:, k] += flag * estimator.predict(X)
                elif k == class_check:
                    y_pred_proba[:, 1] += flag * estimator.predict(X)

                flag = self.learning_rate
                    

        return self.softmax(y_pred_proba)
    
    def predict(self, X):
        """
        Предсказание класса.
        """
        y_pred_proba = self.predict_proba(X)
        return np.argmax(y_pred_proba, axis=1)

In [194]:
X = dataset[dataset.columns[:-1]].to_numpy()[:, [1,3]]
y = dataset["class"].to_numpy()

x_train, x_text, y_train1, y_test1 = train_test_split(X, y, test_size=0.2)


clf = GradientBoosting(max_depth=2, n_estimators=100)
# cols = np.array([1,2,3, 41, 42, 43, 81, 82, 83, 121, 122, 123, 161, 162, 163]) - 1
# clf.fit(X_train[:,:10], y_train.flatten() -1 )
clf.fit(x_train, y_train1 )

otv = clf.predict(x_text)
print(otv)
print(f1_score(y_test1 , otv, average="micro"))

[0 0 0 0 1 1 1 0 0 2 1 2 0 0 0 2 1 0 2 0 0 0 0 1 1 0 1 0 2 1]
0.7


In [135]:
clf.predict_proba(x_text)[:50]

array([[2.55670315e-04, 9.99488659e-01, 2.55670315e-04],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [1.14183646e-21, 1.14183646e-21, 1.00000000e+00],
       [1.14183646e-21, 1.14183646e-21, 1.00000000e+00],
       [2.28875228e-22, 2.28875228e-22, 1.00000000e+00],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [1.14183646e-21, 1.14183646e-21, 1.00000000e+00],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [1.47687904e-01, 1.47687904e-01, 7.04624193e-01],
       [2.55670315e-04, 9.99488659e-01, 2.55670315e-04],
       [1.00000000e+00, 3.72007598e-44, 3.72007598e-44],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333

In [171]:
clf = DecisionTree(max_depth=2)
clf.fit(x_train, y_train1)

otv = clf.predict(x_text)
# otv_prob = clf.predict_proba(x_text)
print(otv)
print(f1_score(y_test1.flatten(), otv, average="micro"))

[1 1 0 1 0 0 1 2 0 0 1 2 2 1 0 1 1 2 0 1 2 2 1 1 1 1 2 1 1 0]
0.9


In [100]:
def AUC_ROC_One_vs_all(y_true, y_pred_proba):
    y_true1, y_pred_proba_1 = list(zip(*list(sorted(zip(y_true, y_pred_proba), key = lambda x : x[1]))))
    print(y_true1)
    print(y_pred_proba_1)

    y_pred_proba_1 = np.where(y_pred_proba_1 > 0.5, 1, 0)
    for i in range(len(y_pred_proba_1)):
        if 

In [101]:
AUC_ROC_One_vs_all(np.array[1,2,3,0,-1,-2], np.array[0.1, 0.7, 0.8, 0.2, 0,3])

TypeError: 'builtin_function_or_method' object is not subscriptable